### PART 1

In [7]:
import csv

In [16]:
# 1-2)
def read_file():
    with open("input_data.csv") as csvfile:
        reader=csv.DictReader(csvfile)
        data = [(row['RECORD_CREATION_DATE'], int(row['LOAN_AMOUNT'])) for row in reader]
        # expanding max function to few lines
        max_loan_amount = max(data, key=lambda x: x[1])[1]
        max_loan_amount_index = map(lambda x:x[1],data).index(max_loan_amount)
        print "RECORD_CREATION_DATE in the file immediately"\
        " preceding the max observed LOAN_AMOUNT:",data[map(lambda x:x[1],data).index(max_loan_amount)-1][0]

In [17]:
read_file()

RECORD_CREATION_DATE in the file immediately preceding the max observed LOAN_AMOUNT: 26May2014:18:03:41.000000


3) If the input data were not sorted by RECORD_CREATION_DATE, how would you solve this problem?
* I would have solved this problem in the same way as I have solved now. However, If it were not sorted then I would have done a sort of RECORD_CREATION_DATE and solved the problem, just to arrange the data in some fashion.

4) How would you modify the max comparison key and zip operation so that your code returns the RECORD_CREATION_DATE before the first observed maximal loan amount? Before the last?
* In case of first observed maximal loan amount, I would have to keep a check that if the next LOAN_AMOUNT is greater than the previous LOAN_AMOUNT. As soon as I see a greater value, I have to stop my iteration and return the date prior to first observed max LOAN_AMOUNT.
* In case of last observed maximal loan amount, Most of the time the max(LOAN_AMOUNT) should give the last maximum LOAN_AMOUNT. If that is not the case, then once we have the max value of loan amount, I will extract the index and check if we more values in the list and if any is greater than the observer max loan amount till now. If we have, replace out max amount with that value and repeat.

### PART 2

In [1]:
import re
import numpy as np
import pandas as pd

In [30]:
# Reading the input file
df = pd.read_csv("input_data.csv")

In [31]:
# 5) Set to a pandas datetime
format = '%d%b%Y:%H:%M:%S.%f'
df["entry_recorded"] = pd.to_datetime(df['RECORD_CREATION_DATE'], format=format)
print df.head()

        RECORD_CREATION_DATE  LOAN_AMOUNT CREDIT_RANGE     EXISTING_DEBT  \
0  01May2014:12:56:37.000000         2500      780-799               NaN   
1  01May2014:16:55:30.000000         4700      760-779   $10,001-$20,000   
2  01May2014:17:05:59.000000         5800      740-759   $20,001-$30,000   
3  02May2014:10:51:57.000000         2500      700-719  $90,001-$100,000   
4  02May2014:14:19:03.000000         4600      780-799   $20,001-$30,000   

                    AGENT_NOTES      OFFICE_LOCATION      entry_recorded  
0    Monthly gross income 1,250     WASHINGTON STATE 2014-05-01 12:56:37  
1  Annual gross income: $15,000  NORTHERN CALIFORNIA 2014-05-01 16:55:30  
2  Annual gross income: $15,000     WASHINGTON STATE 2014-05-01 17:05:59  
3    Monthly gross income 5,480  SOUTHERN CALIFORNIA 2014-05-02 10:51:57  
4                           NaN     WASHINGTON STATE 2014-05-02 14:19:03  


In [32]:
# 6-10) Returns monthly net income if it appears in the AGENT_NOTES field (divide annual income by 12)
def monthly_income_parser(x):
    m_amount = 0
    a_amount = 0
    month_match = False
    annual_match = False
    if str(x).find('nan') > -1:
        return np.nan

    if x.find('Monthly gross income') > -1:
        month_match = True
        m_amount_lst = re.findall('\d+,*\d*', x)
        if len(m_amount_lst) > 1:
            # utility function
            amount_lst = monthly_income_parser_util(m_amount_lst)
            m_amount = max(amount_lst) - min(amount_lst)
            m_amount = '$'+str(m_amount)
        else:
            num = ''.join(filter(lambda x:  x.isdigit(), m_amount_lst[0]))
            m_amount = '$'+num

    elif x.find('Annual gross income') > -1:
        annual_match = True
        m_amount_annual_lst = re.findall('\d+,*\d*', x)
        if len(m_amount_annual_lst) > 1:
            # utility function
            amount_lst = monthly_income_parser_util(m_amount_annual_lst)
            m_income = max(amount_lst)//12
            a_amount = m_income - min(amount_lst)
            a_amount = '$' + str(a_amount)
        else:
            num = int(''.join(filter(lambda x:  x.isdigit(), m_amount_annual_lst[0])))
            a_amount = num//12
            a_amount = '$' + str(a_amount)

    if month_match:
        return m_amount
    elif annual_match:
        return a_amount
    else:
        return np.nan

# Utility function to parse income as int
def monthly_income_parser_util(m_amount_lst):
    amount_lst = []
    for each in m_amount_lst:
        num = int(''.join(filter(lambda x:  x.isdigit(), each)))
        amount_lst.append(num)
    return amount_lst

df["monthly_income"] = df["AGENT_NOTES"].apply(monthly_income_parser)
print df.head()

        RECORD_CREATION_DATE  LOAN_AMOUNT CREDIT_RANGE     EXISTING_DEBT  \
0  01May2014:12:56:37.000000         2500      780-799               NaN   
1  01May2014:16:55:30.000000         4700      760-779   $10,001-$20,000   
2  01May2014:17:05:59.000000         5800      740-759   $20,001-$30,000   
3  02May2014:10:51:57.000000         2500      700-719  $90,001-$100,000   
4  02May2014:14:19:03.000000         4600      780-799   $20,001-$30,000   

                    AGENT_NOTES      OFFICE_LOCATION      entry_recorded  \
0    Monthly gross income 1,250     WASHINGTON STATE 2014-05-01 12:56:37   
1  Annual gross income: $15,000  NORTHERN CALIFORNIA 2014-05-01 16:55:30   
2  Annual gross income: $15,000     WASHINGTON STATE 2014-05-01 17:05:59   
3    Monthly gross income 5,480  SOUTHERN CALIFORNIA 2014-05-02 10:51:57   
4                           NaN     WASHINGTON STATE 2014-05-02 14:19:03   

  monthly_income  
0          $1250  
1          $1250  
2          $1250  
3         

In [33]:
# 11) Set to be equal to the lower bound of the credit score range.
def credit_score_format(x):
    if x.find('-') > -1:
        return x.split('-')[0]
    else:
        return x

df["credit_score"] = df["CREDIT_RANGE"].apply(credit_score_format)
print df.head()

        RECORD_CREATION_DATE  LOAN_AMOUNT CREDIT_RANGE     EXISTING_DEBT  \
0  01May2014:12:56:37.000000         2500      780-799               NaN   
1  01May2014:16:55:30.000000         4700      760-779   $10,001-$20,000   
2  01May2014:17:05:59.000000         5800      740-759   $20,001-$30,000   
3  02May2014:10:51:57.000000         2500      700-719  $90,001-$100,000   
4  02May2014:14:19:03.000000         4600      780-799   $20,001-$30,000   

                    AGENT_NOTES      OFFICE_LOCATION      entry_recorded  \
0    Monthly gross income 1,250     WASHINGTON STATE 2014-05-01 12:56:37   
1  Annual gross income: $15,000  NORTHERN CALIFORNIA 2014-05-01 16:55:30   
2  Annual gross income: $15,000     WASHINGTON STATE 2014-05-01 17:05:59   
3    Monthly gross income 5,480  SOUTHERN CALIFORNIA 2014-05-02 10:51:57   
4                           NaN     WASHINGTON STATE 2014-05-02 14:19:03   

  monthly_income credit_score  
0          $1250          780  
1          $1250      

In [35]:
# 12) Set to be equal to the lower bound of the debt range
def lower_debt_range(x):
    if str(x).find('-') > -1:
        return x.split('-')[0]
    elif x == 'NaN':
        return np.NaN
    else:
        return x
df["debt"] = df["EXISTING_DEBT"].apply(lower_debt_range)
print df.head()

        RECORD_CREATION_DATE  LOAN_AMOUNT CREDIT_RANGE     EXISTING_DEBT  \
0  01May2014:12:56:37.000000         2500      780-799               NaN   
1  01May2014:16:55:30.000000         4700      760-779   $10,001-$20,000   
2  01May2014:17:05:59.000000         5800      740-759   $20,001-$30,000   
3  02May2014:10:51:57.000000         2500      700-719  $90,001-$100,000   
4  02May2014:14:19:03.000000         4600      780-799   $20,001-$30,000   

                    AGENT_NOTES      OFFICE_LOCATION      entry_recorded  \
0    Monthly gross income 1,250     WASHINGTON STATE 2014-05-01 12:56:37   
1  Annual gross income: $15,000  NORTHERN CALIFORNIA 2014-05-01 16:55:30   
2  Annual gross income: $15,000     WASHINGTON STATE 2014-05-01 17:05:59   
3    Monthly gross income 5,480  SOUTHERN CALIFORNIA 2014-05-02 10:51:57   
4                           NaN     WASHINGTON STATE 2014-05-02 14:19:03   

  monthly_income credit_score     debt  
0          $1250          780      NaN  
1   

13) What is the advantage of using built-in pandas string methods as opposed to apply()?
* The built-in pandas string method makes it easy to operate on each element of the array. They also handle all mising automatically. We also get many of string methods such as str.split(), str.contains() etc., which are really helpful in processing and transforming data
* Apply() works on entire input axis of DataFrame i.e. row/column. Also, it cannot handle missing value automatically.

In [36]:
# 14) Average number of hours between record creations.
l = len(df.entry_recorded)-1
for i in xrange(l):
    df['time_diff'] = df.entry_recorded[i+1] - df.entry_recorded[i]
sum = df['time_diff'].sum()
avg = sum/len(df['time_diff'])
print "Average number of hours between record creations:",avg

Average number of hours between record creations: 0 days 02:24:53


In [37]:
# 15) How many records are from california office?
print "Records are from california office: ",len(df[df['OFFICE_LOCATION'].str.contains("CALIFORNIA")])

Records are from california office:  2405
